In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install flaml
!pip install mplcyberpunk

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import BatchNormalization, Dense, Dropout
from sklearn.model_selection import train_test_split
from flaml import AutoML
import seaborn as sns
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import mplcyberpunk

In [ ]:
TaitanicTrain = pd.read_csv(r'/kaggle/input/spaceship-titanic/train.csv')

TaitanicTest = pd.read_csv(r'/kaggle/input/spaceship-titanic/test.csv')

submitData = pd.read_csv(r'/kaggle/input/spaceship-titanic/sample_submission.csv')

In [ ]:
TrainData = TaitanicTrain.fillna(0)
TestData = TaitanicTest.fillna(0)

TrainData = TrainData.loc[:, ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]
TestData = TestData.loc[:, ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]

In [ ]:
TrainData

In [ ]:
X = np.array(TrainData)
y = np.asarray(TaitanicTrain.loc[:, 'Transported']).astype('int64')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = AutoML()

model.fit(X_train, y_train, task='classification', time_budget=20, metric='accuracy')

In [ ]:
print('Best ML Model:', model.best_estimator)
print('Best hyperparmeter config:', model.best_config)
print('Best Accuracy on validation data: %f'%(1 - model.best_loss))
print('Training duration of best run: %f s'%(model.best_config_train_time))

In [ ]:
class BaseModel(tf.keras.Model):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.D1 = Dense(32, activation='relu')
        self.B1 = BatchNormalization()
        self.D2 = Dense(64)
        self.Dr1 = Dropout(0.3)
        self.D3 = Dense(2, activation='sigmoid')
        
    def call(self, x):
        x = self.D1(x)
        x = self.B1(x)
        x = self.D2(x)
        x = self.Dr1(x)
        y = self.D3(x)
        return y
    
TFmodel = BaseModel()

TFmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['sparse_categorical_accuracy'])

history = TFmodel.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

TFmodel.summary()

In [ ]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 6))
plt.style.use('cyberpunk')
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Acc')
plt.plot(val_acc, label='Validation Acc')
plt.title('Training And Validation Acc')
plt.legend()
mplcyberpunk.add_glow_effects()


plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training And Validation Loss')
plt.legend()
mplcyberpunk.add_glow_effects()
plt.show()

In [ ]:
max(val_acc), max(acc)

In [ ]:
X_new = np.array(TestData)

TFpred = TFmodel.predict(X_new)

submitData.Transported = tf.argmax(TFpred, axis=1)

In [ ]:
submitData

In [ ]:
submitData.to_csv('Titanicsubmit.csv', index=False)